In [10]:
from pydantic import BaseModel, field_validator, ValidationError, ValidationInfo, BeforeValidator, AfterValidator

In [2]:
class Model(BaseModel):
    field_1: int
    field_2: list[int]
    field_3: str
    field_4: list[str]

    @field_validator('field_3')
    @classmethod
    def validate(cls, value: str, validated_values: ValidationInfo):
        print(f"{value=}")
        print(f"{validated_values=}")
        return value

In [3]:
Model(field_1=1, field_2=[1, 2, 3], field_3="abc", field_4=["a", "b", "c"])

value='abc'
validated_values=ValidationInfo(config={'title': 'Model'}, context=None, data={'field_1': 1, 'field_2': [1, 2, 3]}, field_name='field_3')


Model(field_1=1, field_2=[1, 2, 3], field_3='abc', field_4=['a', 'b', 'c'])

In [6]:
try:
    Model(field_1=1, field_2=["a", "f"], field_3="abc", field_4=["a", "b", "c"])
except ValidationError as e:
    print(e)

value='abc'
validated_values=ValidationInfo(config={'title': 'Model'}, context=None, data={'field_1': 1}, field_name='field_3')
2 validation errors for Model
field_2.0
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='a', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing
field_2.1
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='f', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing


In [7]:
import pytz
from dateutil.parser import parse
from typing import Any
from datetime import datetime


def parse_datetime(v: Any) -> datetime:
    if isinstance(v, str):
        try:
            return parse(v)
        except Exception as e:
            raise ValueError(str(e))
    return v


def validate_date(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt

In [11]:
from typing import Annotated

DateTimeUTC = Annotated[datetime, BeforeValidator(parse_datetime), AfterValidator(parse_datetime)]

In [12]:
class Model(BaseModel):
    start: DateTimeUTC
    end: DateTimeUTC

    @field_validator('end')
    @classmethod
    def validate(cls, value: datetime, validated_values: ValidationInfo):
        data = validated_values.data
        if "start" in data and value <= data["start"]:
                raise ValueError("Start date must be before end date")
        return value

In [13]:
Model(start="2020/1/1", end="2020/2/1")

Model(start=datetime.datetime(2020, 1, 1, 0, 0), end=datetime.datetime(2020, 2, 1, 0, 0))

In [14]:
try:
    Model(start="2020/10/1", end="2020/2/1")
except ValidationError as e:
    print(e)

1 validation error for Model
end
  Value error, Start date must be before end date [type=value_error, input_value='2020/2/1', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


In [15]:
Model(end="2020/2/1")

ValidationError: 1 validation error for Model
start
  Field required [type=missing, input_value={'end': '2020/2/1'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing